In [1]:
import pandas as pd
import datetime
from numpy import inf

In [98]:
route_id = 85
date = datetime.datetime.strptime('03/20/2020', '%m/%d/%Y')
session_id = 1
data = {
    'route_supervisor': '',
    'route_name': '',
    'date': '',
    'vehicle_number': '',
    'session': '',
    'sales': {},
    'milk': {},
    'fermented': {}
}
# get vehicle details
data['vehicle_number'] = RouteVehicleMap.objects.get(route_id=route_id).vehicle.licence_number
data['route_supervisor'] = RouteVehicleMap.objects.get(route_id=route_id).vehicle.driver_name
data['date'] = date
data['session'] = Session.objects.get(id=session_id).display_name

route = Route.objects.get(id=route_id)
leak_percentage = route.leak_packet_in_percentage
data['route_name'] = route.name
business_ids = RouteBusinessMap.objects.filter(route=route).values_list('business', flat=True)
sale_group = SaleGroup.objects.filter(business_id__in=business_ids, date=date, session_id=session_id)
sale_values = Sale.objects.filter(sale_group__in=sale_group).values_list('product_id', 'count')
sale_columns = ['product_id', 'count']
sale_df = pd.DataFrame(list(sale_values), columns=sale_columns)

icustomer_sale_group = ICustomerSaleGroup.objects.filter(business_id__in=business_ids, date__month=date.month, date__year=date.year,session_id=session_id)
icustomer_sale_values = ICustomerSale.objects.filter(icustomer_sale_group__in=icustomer_sale_group).values_list('product_id', 'count')
icustomer_sale_columns = ['product_id', 'count']
icustomer_sale_df = pd.DataFrame(list(icustomer_sale_values), columns=icustomer_sale_columns)

combined_df = sale_df.merge(icustomer_sale_df, how="outer")

product_values = Product.objects.filter(is_active=True).order_by('display_ordinal').values_list('id', 'short_name', 'unit', 'quantity')
product_columns = ['id', 'product_short_name', 'unit', 'quantity']
product_df = pd.DataFrame(list(product_values), columns=product_columns)
total_sale_df = combined_df.groupby(['product_id'])['count'].sum().reset_index()
merged_df = pd.merge(total_sale_df, product_df, how='left', left_on='product_id', right_on='id')
data['sales'] = merged_df.groupby('id').apply(lambda x: x.to_dict('r')[0]).to_dict()
milk_product_ids = [1, 2, 3, 4, 6]
# find milk total and net total
data['milk_total_count'] = merged_df[merged_df['id'].isin(milk_product_ids)]['count'].sum()
data['net_total_count'] = merged_df['count'].sum() 
tray_config_values = ProductTrayConfig.objects.all().values_list('product_id', 'tray_count', 'product_count')
tray_config_columns = ['product_id', 'p_c_tray_count', 'p_c_product_count']
tray_config_df = pd.DataFrame(list(tray_config_values), columns=tray_config_columns)
tray_config_merge_df = pd.merge(merged_df, tray_config_df, how='left', left_on='id', right_on='product_id')
tray_config_merge_df['calculated_tray_count'] = tray_config_merge_df.apply(
            lambda x: findout_tray_count(x['count'], x['p_c_product_count']), axis=1)
tray_config_merge_df['calculated_pocket_count'] = tray_config_merge_df.apply(
    lambda x: findout_packet_count(x['count'], x['p_c_product_count']), axis=1)
tray_config_merge_df['calculated_pocket_count_in_negative'] = tray_config_merge_df.apply(
    lambda x: findout_packet_count_in_negative(x['count'], x['p_c_product_count'], x['calculated_tray_count']), axis=1)
tray_config_merge_df = tray_config_merge_df.fillna(0)
tray_config_merge_df['count'] = tray_config_merge_df['count'].astype(int)
tray_config_merge_df['calculated_pocket_count'] = tray_config_merge_df['calculated_pocket_count'].astype(int)
tray_config_merge_df['litre'] = (tray_config_merge_df['quantity'] / 1000) * (tray_config_merge_df['count'] + tray_config_merge_df['calculated_pocket_count'])
tray_config_merge_df = tray_config_merge_df[['id', 'count', 'quantity','calculated_tray_count', 'calculated_pocket_count', 'product_short_name', 'calculated_pocket_count_in_negative']]
tray_config_merge_df['count'] = tray_config_merge_df['count'].astype(int)
tray_config_merge_df['leak_packet'] = (leak_percentage * tray_config_merge_df['count'])
tray_config_merge_df['leak_packet'] = tray_config_merge_df['leak_packet'].astype(int)
allowance_business_ids= list(BusinessWiseLeakageAllowanceAsPacket.objects.all().values_list('business_id', flat=True))
if RouteBusinessMap.objects.filter(route_id=route.id, business_id__in=allowance_business_ids).exists():
    route_business_ids = list(RouteBusinessMap.objects.filter(route_id=route.id).values_list('business_id', flat=True))
    allowance_business_obj = BusinessWiseLeakageAllowanceAsPacket.objects.filter(business_id__in=route_business_ids)
    for allowance_business in allowance_business_obj:
        print(allowance_business.product.id)
        print(len(tray_config_merge_df[tray_config_merge_df['id'] == allowance_business.product.id]))
        if not len(tray_config_merge_df[tray_config_merge_df['id'] == allowance_business.product.id]) == 0:
            tray_config_merge_df.loc[tray_config_merge_df['id'] == allowance_business.product.id, 'leak_packet'] = tray_config_merge_df['leak_packet'] + allowance_business.packet_count
        else:
            new_list = [allowance_business.product.id, '0', allowance_business.product.quantity, '0', '0',allowance_business.product.short_name,'0', allowance_business.packet_count]
            to_index = len(tray_config_merge_df)
            tray_config_merge_df.loc[to_index] = new_list
tray_config_merge_df


2
1


,id,count,quantity,calculated_tray_count,calculated_pocket_count,product_short_name,calculated_pocket_count_in_negative,leak_packet
0,1,1202,500.000,50.0,2,TM 500,0.0,2
1,2,2002,500.000,83.0,10,STD 500,0.0,84
2,3,2,250.000,0.0,2,STD 250,0.0,0
3,4,1,500.000,0.0,1,FCM 500,0.0,0
4,6,2502,1000.000,208.0,0,FCM 1000,6.0,5
5,25,1400,500.000,0.0,0,Curd 500,0.0,2


In [1]:
def findout_packet_count_in_negative(count, product_defalut_count, tray_count):
    if not tray_count == 0:
        if not product_defalut_count == 0:
            remainder_of_tray_count = count % product_defalut_count
            coefieient_of_tray_count = count // product_defalut_count
        else:
            remainder_of_tray_count = 0
            coefieient_of_tray_count = 0
        if remainder_of_tray_count >= (product_defalut_count / 2):
            return product_defalut_count - remainder_of_tray_count
        else:
            return 0
    else:
        return 0


In [2]:
len(tray_config_merge_df[tray_config_merge_df['id'] == 99])

NameError: name 'tray_config_merge_df' is not defined

In [3]:
print(Product.objects.filter(group_id=2).values_list('short_name', flat=True))

Product.objects.filter(group_id=2).values_list('id', flat=True)

<QuerySet ['Curd 500', 'Curd 150', 'BM 200', 'CBUK', 'Lassi 200', 'Curd 100']>


<QuerySet [25, 7, 8, 21, 9, 10]>

In [4]:
def findout_tray_count(count, product_defalut_count):
    if not product_defalut_count == 0:
        remainder_of_tray_count = count % product_defalut_count
        coefieient_of_tray_count = count // product_defalut_count
    else:
        remainder_of_tray_count = 0
        coefieient_of_tray_count = 0
    if not remainder_of_tray_count == 0:
        if remainder_of_tray_count > (product_defalut_count / 2):
            return coefieient_of_tray_count + 1
        else:
            return coefieient_of_tray_count
    else:
        return coefieient_of_tray_count

In [5]:
def findout_packet_count(count, product_defalut_count):
    if not product_defalut_count == 0:
        remainder_of_tray_count = count % product_defalut_count
        coefieient_of_tray_count = count // product_defalut_count
    else:
        remainder_of_tray_count = 0
        coefieient_of_tray_count = 0
    if not remainder_of_tray_count == 0:
        if remainder_of_tray_count < (product_defalut_count / 2):
            return remainder_of_tray_count
        else:
            return 0
    else:
        return remainder_of_tray_count

In [8]:
findout_packet_count_in_negative(346778, 48, 7225)
# findout_tray_count(346778, 48)
# get_business_ids(85, 1, '2020-03-20')

22

In [124]:
def get_business_ids(route_id,session_id, date):
    business_ids = RouteBusinessMap.objects.filter(route_id=route_id).values_list('business_id', flat=True)
    ordered_business = list(SaleGroup.objects.filter(business_id__in=business_ids,date=date,session_id=session_id).values_list('business_id', flat=True))
    selected_month = datetime.datetime.strptime(date, '%Y-%m-%d')
    ordered_business_for_icustomer = list(ICustomerSaleGroup.objects.filter(business_id__in=business_ids, date__month=selected_month.month, date__year=selected_month.year, session_id=session_id).values_list('business_id', flat=True))
    return list(set(ordered_business + ordered_business_for_icustomer))


In [16]:
list(RouteBusinessMap.objects.filter(route__name='J - RAMANA_MOR').values_list('business__code'))

[('13',),
 ('14',),
 ('83',),
 ('92',),
 ('105',),
 ('112',),
 ('122',),
 ('156',),
 ('240',),
 ('276',),
 ('285',),
 ('296',),
 ('417',),
 ('418',),
 ('419',),
 ('437',),
 ('447',),
 ('458',),
 ('470',),
 ('481',),
 ('514',),
 ('1038',),
 ('1082',),
 ('1233',),
 ('1883',),
 ('1892',),
 ('1945',),
 ('1958',),
 ('1969',),
 ('2017',),
 ('2157',),
 ('2208',),
 ('2210',),
 ('2211',),
 ('2293',),
 ('2302',),
 ('2321',),
 ('2375',),
 ('2382',),
 ('2396',),
 ('3034',),
 ('3035',),
 ('3148',),
 ('4191',),
 ('4197',),
 ('4215',),
 ('2577',)]

In [83]:
def selected_date_range(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [118]:
end_date = datetime.datetime.now()
start_date = end_date - timedelta(days=7)
sale_obj = Sale.objects.filter(sale_group__date__gte=start_date, sale_group__date__lte=end_date,
                                   product__group_id=1)
sale_list = list(
    sale_obj.values_list('id', 'sale_group', 'sale_group__business_id', 'sale_group__business__business_type_id',
                         'sale_group__date', 'sale_group__session',
                         'sale_group__session__display_name', 'sale_group__ordered_via',
                         'sale_group__ordered_via__name', 'sale_group__payment_status__name',
                         'sale_group__sale_status__name', 'sale_group__total_cost', 'product__group_id',
                         'product__group__name', 'product__quantity', 'product',
                         'product__name', 'count', 'cost'))
sale_column = ['sale_id', 'sale_group_id', 'business_id', 'business_type_id', 'date', 'session_id',
               'session_name',
               'ordered_via_id', 'ordered_via_name', 'payment_status',
               'sale_status',
               'session_wise_price', 'product_group_id', 'product_group_name', 'product_default_quantity',
               'product_id', 'product_name', 'count', 'product_cost']
sale_df = pd.DataFrame(sale_list, columns=sale_column)


final_icustomer_df = pd.DataFrame(columns=['sale_id', 'sale_group_id', 'business_id', 'business_type_id', 'date', 'session_id', 'session_name',
               'ordered_via_id', 'ordered_via_name', 'payment_status',
               'sale_status',
               'session_wise_price', 'product_group_id', 'product_default_quantity', 'product_id', 'product_name',
               'count', 'product_cost'])
for single_date in selected_date_range(start_date + timedelta(days=1), end_date + timedelta(days=1)):
    date = single_date.strftime("%Y-%m-%d")
    requested_month = datetime.datetime.strptime(date, '%Y-%m-%d').month
    icustomer_sale_obj = ICustomerSale.objects.filter(icustomer_sale_group__date__month=requested_month)
    icustomer_sale_list = list(
        icustomer_sale_obj.values_list('id', 'icustomer_sale_group', 'icustomer_sale_group__business_id', 'icustomer_sale_group__business__business_type_id',
                             'icustomer_sale_group__date', 'icustomer_sale_group__session',
                             'icustomer_sale_group__session__display_name', 'icustomer_sale_group__ordered_via',
                             'icustomer_sale_group__ordered_via__name', 'icustomer_sale_group__payment_status__name',
                             'icustomer_sale_group__sale_status__name', 'icustomer_sale_group__total_cost',
                             'product__group_id','product__group__name', 'product__quantity', 'product',
                             'product__name', 'count', 'cost'))
    icustomer_sale_column = ['sale_id', 'sale_group_id', 'business_id', 'business_type_id', 'date', 'session_id', 'session_name',
                   'ordered_via_id', 'ordered_via_name', 'payment_status',
                   'sale_status',
                   'session_wise_price', 'product_group_id', 'product_group_name','product_default_quantity', 'product_id', 'product_name',
                   'count', 'product_cost']

    icustomer_sale_df = pd.DataFrame(icustomer_sale_list, columns=icustomer_sale_column)
    icustomer_sale_df['date'] = date
    final_icustomer_df = final_icustomer_df.append(icustomer_sale_df)
final_df = sale_df.merge(final_icustomer_df, how="outer")

final_df['count'] = final_df['count'].astype('float')
final_df['date'] = final_df['date'].astype('str')
final_df['product_default_quantity'] = final_df['product_default_quantity'].astype('float')
final_df['count_multiply_by_quantity'] = final_df['product_default_quantity'] * final_df['count'] / 1000
master_dict_product_group = {'data': []}
for single_date in selected_date_range(start_date + timedelta(days=1), end_date + timedelta(days=1)):
    date = single_date.strftime("%Y-%m-%d")
    print(date)
    filtered_df = final_df[final_df['date'] == date]
    if not filtered_df.empty:
        master_dict_product_group['data'].append(filtered_df['count_multiply_by_quantity'].sum())
    else:
        master_dict_product_group['data'].append(0)
master_dict_product_group
# final_icustomer_df['count'] = final_icustomer_df['count'].astype('float')
# final_icustomer_df['date'] = final_icustomer_df['date'].astype('str')
# final_icustomer_df['product_default_quantity'] = final_icustomer_df['product_default_quantity'].astype('float')
# final_icustomer_df['count_multiply_by_quantity'] = final_icustomer_df['product_default_quantity'] * final_icustomer_df['count'] / 1000
# final_icustomer_df


2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17


{'data': [3.75, 3.75, 3.75, 3.75, 3.75, 63.75, 15.75]}

In [79]:
empty_list = []
test_df = pd.DataFrame(columns=['sale_id', 'sale_group_id', 'business_id', 'business_type_id', 'date', 'session_id', 'session_name',
               'ordered_via_id', 'ordered_via_name', 'payment_status',
               'sale_status',
               'session_wise_price', 'product_group_id', 'product_default_quantity', 'product_id', 'product_name',
               'count', 'product_cost'])
for i in range(1, 4):
    icustomer_sale_obj = ICustomerSale.objects.filter(icustomer_sale_group__date__month=requested_month)
    customer_sale_list = list(
        icustomer_sale_obj.values_list('id', 'icustomer_sale_group', 'icustomer_sale_group__business_id', 'icustomer_sale_group__business__business_type_id',
                             'icustomer_sale_group__date', 'icustomer_sale_group__session',
                             'icustomer_sale_group__session__display_name', 'icustomer_sale_group__ordered_via',
                             'icustomer_sale_group__ordered_via__name', 'icustomer_sale_group__payment_status__name',
                             'icustomer_sale_group__sale_status__name', 'icustomer_sale_group__total_cost',
                             'product__group_id', 'product__quantity', 'product',
                             'product__name', 'count', 'cost'))
    icustomer_sale_column = ['sale_id', 'sale_group_id', 'business_id', 'business_type_id', 'date', 'session_id', 'session_name',
                   'ordered_via_id', 'ordered_via_name', 'payment_status',
                   'sale_status',
                   'session_wise_price', 'product_group_id', 'product_default_quantity', 'product_id', 'product_name',
                   'count', 'product_cost']

    icustomer_sale_df = pd.DataFrame(icustomer_sale_list, columns=icustomer_sale_column)
    test_df = test_df.append(icustomer_sale_df)
test_df


,sale_id,sale_group_id,business_id,business_type_id,date,session_id,session_name,ordered_via_id,ordered_via_name,payment_status,sale_status,session_wise_price,product_group_id,product_default_quantity,product_id,product_name,count,product_cost
0,48,34,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,103.00,1,1000.000,6,FCM,1.0,49.00
1,45,34,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,103.00,1,500.000,1,Tonned,1.0,20.00
2,47,34,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,103.00,1,250.000,3,STD,1.0,11.50
3,46,34,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,103.00,1,500.000,2,STD,1.0,22.50
4,41,32,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,54.00,1,500.000,1,Tonned,1.0,20.00
5,43,32,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,54.00,1,250.000,3,STD,1.0,11.50
6,42,32,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,54.00,1,500.000,2,STD,1.0,22.50
7,44,33,2569,1,2020-03-01,2,Eve,2,Portal,Paid,Ordered,11.50,1,250.000,3,STD,1.0,11.50
0,48,34,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,103.00,1,1000.000,6,FCM,1.0,49.00
1,45,34,2569,1,2020-03-01,1,Mor,2,Portal,Paid,Ordered,103.00,1,500.000,1,Tonned,1.0,20.00


In [147]:
product_values = Product.objects.filter(is_active=True).values_list('id', 'short_name', 'unit__name', 'quantity')
product_columns = ['id', 'product_short_name', 'product_unit', 'quantity']
product_df = pd.DataFrame(list(product_values), columns=product_columns)
tray_config_values = ProductTrayConfig.objects.all().values_list('product_id', 'tray_count', 'product_count')
tray_config_columns = ['tray_product_id', 'p_c_tray_count', 'p_c_product_count']
tray_config_df = pd.DataFrame(list(tray_config_values), columns=tray_config_columns)
tray_config_df = tray_config_df.fillna(0)
product_tray_merge_df = pd.merge(product_df, tray_config_df, how='left', left_on='id', right_on='tray_product_id')
product_tray_merge_df = product_tray_merge_df.fillna(0)
business_code = '2600'
session_id = 1
month = datetime.datetime.strptime('2020-03-06', '%Y-%m-%d').month
date = '2020-03-16'

business = Business.objects.get(code=business_code)
route = RouteBusinessMap.objects.filter(business=business)[0].route
agent = BusinessAgentMap.objects.filter(business=business)[0].agent
session = Session.objects.get(id=session_id)
data = {
    'agent_first_name': agent.user_profile.user.first_name,
    'agent_last_name': agent.user_profile.user.last_name,
    'agent_code': agent.agent_code,
    'route_name': route.name,
    'date': date,
    'session': session.display_name,
    'business_code': business.code,
    'no': '232423',
    'total_cost': None,
    'sales': None
}
sale_group = SaleGroup.objects.filter(date=date, session_id=session_id, business__code=business_code)
business_sale_group = sale_group[0]
sales_values = Sale.objects.filter(sale_group__in=sale_group).values_list('sale_group', 'product_id', 'count')
sales_columns = ['sale_group_id', 'product_id', 'count']
sale_df = pd.DataFrame(list(sales_values), columns=sales_columns)
sale_df = sale_df.fillna(0)
data['total_cost'] = sale_group.aggregate(Sum('total_cost'))['total_cost__sum']
final_df = pd.merge(sale_df, product_tray_merge_df, how='left', left_on='product_id', right_on='id')
final_df = final_df.fillna(0)
business_df = final_df

icustomer_sale_obj = ICustomerSale.objects.filter(icustomer_sale_group__date__month=month,icustomer_sale_group__session_id=session_id, icustomer_sale_group__business__code=business_code)
icustomer_sales_values = list(icustomer_sale_obj.values_list('icustomer_sale_group', 'product_id', 'count', 'icustomer_sale_group__icustomer'))
icustomer_sales_columns = ['icustomer_sale_group_id', 'product_id', 'indivioul_icustomer_count', 'icustomer_id']
icustomer_sale_df = pd.DataFrame(icustomer_sales_values, columns=icustomer_sales_columns)
icustomer_sale_df['icustomer_count'] = icustomer_sale_df.groupby(['product_id'])[
        'indivioul_icustomer_count'].transform('sum')
icustomer_sale_df = icustomer_sale_df.drop_duplicates(subset=['product_id'], keep="first")
icustomer_sale_df = icustomer_sale_df.fillna(0)
icustomer_sale_df = pd.merge(icustomer_sale_df, product_tray_merge_df, how='left', left_on='product_id', right_on='id')
# construct business sale df
new_df = pd.merge(business_df, icustomer_sale_df, how='outer',
                  left_on=['id', 'product_short_name', 'p_c_product_count', 'p_c_tray_count', 'product_unit'],
                  right_on=['id', 'product_short_name', 'p_c_product_count', 'p_c_tray_count', 'product_unit'])
new_df = new_df[['id', 'count', 'icustomer_count', 'product_short_name', 'p_c_product_count', 'p_c_tray_count', 'product_unit']]
new_df['count'] = new_df['count'].fillna(0)
new_df['icustomer_count'] = new_df['icustomer_count'].fillna(0)
new_df['count'].astype(int)
new_df['icustomer_count'].astype(int)
new_df['calculated_tray_count'] = ((new_df['count'] + new_df['icustomer_count']) / new_df['p_c_product_count']) * new_df['p_c_tray_count']
new_df['calculated_tray_count'] = new_df['calculated_tray_count'].fillna(0)
new_df['calculated_tray_count'] = new_df['calculated_tray_count'].astype(int)
new_df['calculated_pocket_count'] = (new_df['count'] + new_df['icustomer_count']) % new_df['p_c_product_count']
new_df = new_df.fillna(0)
data['sales'] = new_df.groupby('id').apply(lambda x: x.to_dict('r')[0]).to_dict()
data['total_business_product_count'] = new_df['count'].sum()
data['total_icustomer_product_count'] = new_df['icustomer_count'].sum()
data


{'agent_first_name': 'Nan',
 'agent_last_name': 'Nan',
 'agent_code': '2568',
 'route_name': '1 - SULUR_MOR',
 'date': '2020-03-16',
 'session': 'Mor',
 'business_code': '2600',
 'no': '232423',
 'total_cost': Decimal('2514.00'),
 'sales': {1: {'id': 1,
   'count': 12.0,
   'icustomer_count': 2.0,
   'product_short_name': 'TM 500',
   'p_c_product_count': 24.0,
   'p_c_tray_count': 1.0,
   'product_unit': 'ml',
   'calculated_tray_count': 0,
   'calculated_pocket_count': 14.0},
  2: {'id': 2,
   'count': 0.0,
   'icustomer_count': 2.0,
   'product_short_name': 'STD 500',
   'p_c_product_count': 24.0,
   'p_c_tray_count': 1.0,
   'product_unit': 'ml',
   'calculated_tray_count': 0,
   'calculated_pocket_count': 2.0},
  3: {'id': 3,
   'count': 120.0,
   'icustomer_count': 2.0,
   'product_short_name': 'STD 250',
   'p_c_product_count': 48.0,
   'p_c_tray_count': 1.0,
   'product_unit': 'ml',
   'calculated_tray_count': 2,
   'calculated_pocket_count': 26.0},
  4: {'id': 4,
   'count': 1

In [144]:
BusinessAgentMap.objects.get(business__code='2600').agent.user_profile.user.first_name

'Nan'

In [13]:
sale_group_obj = ICustomerSaleGroup.objects.filter(icustomer_id=icustomer_id).order_by('-id')
sale_group_list = list(sale_group_obj.values_list('id', 'business_id', 'business__code',
                                      'business__business_type_id', 'icustomer_id',
                                      'icustomer__customer_code',
                                      'date', 'session',
                                      'session__display_name', 'ordered_via',
                                      'ordered_via__name', 'payment_status__name',
                                      'sale_status__name', 'total_cost','total_cost_for_month','time_created'))
sale_group_column = ['sale_group_id', 'business_id', 'business_code', 'business_type_id', 'icustomer_id',
               'icustomer_code', 'date', 'session_id', 'session_name',
               'ordered_via_id', 'ordered_via_name','payment_status',
               'sale_status',
               'session_wise_price','total_cost_for_month','order_date']
sale_group_df = pd.DataFrame(sale_group_list, columns=sale_group_column)
sale_group_df['month'] = pd.DatetimeIndex(sale_group_df['date']).month
sale_group_df['year'] = pd.DatetimeIndex(sale_group_df['date']).year
sale_group_df['total_price_per_month'] = sale_group_df.groupby(['month','year'])[
        'total_cost_for_month'].transform('sum')
sale_group_df = sale_group_df.drop_duplicates(subset=['month', 'year'], keep="first")
sale_group_df

,sale_group_id,business_id,business_code,business_type_id,icustomer_id,icustomer_code,date,session_id,session_name,ordered_via_id,ordered_via_name,payment_status,sale_status,session_wise_price,total_cost_for_month,order_date,month,year,total_price_per_month
0,10,1848,623,1,11,1234,2020-05-01,2,Eve,2,Portal,Paid,Ordered,85.00,2635.00,2020-03-13 06:41:53.469340+00:00,5,2020,5068.50
2,8,1848,623,1,11,1234,2020-04-01,2,Eve,2,Portal,Paid,Ordered,91.50,2745.00,2020-03-13 06:39:26.318920+00:00,4,2020,4365.00


In [161]:
date = '3/16/2020'
datetime.datetime.strptime(date, '%m/%d/%Y').

3